In [2]:
#Point 1
import sqlite3

conn = sqlite3.connect("db.db")

cursor = conn.cursor()

# with open("myshop.sql","r") as f: -- MYSQL format not accepter
with open("mysqop_lite.sql","r") as f:
    sql_query = f.read()

cursor.executescript(sql_query)#can execute many SQL queries (separated at ;)
conn.commit()
cursor.close()
conn.close()



In [49]:
#Point 4
from sqlalchemy import create_engine,text
import pymysql
import pandas as pd

cities = pd.read_excel("Cities.xlsx")

engine = create_engine('mysql+pymysql://root:@localhost/geography')

with engine.begin() as con:
    countries_ids = pd.read_sql(text("SELECT id,external_id FROM countries"),con)
    existing_ext_ids = pd.read_sql(text("SELECT ext_id FROM cities"),con)

mask = cities["EXT_ID"].isin(existing_ext_ids["ext_id"])
cities = cities[~mask]
cities = cities.merge(countries_ids,left_on="COUNTRY_EXT_ID",right_on="external_id")

col_ren = {"id":"country"}
cities.columns = [col_ren.get(x,x) for x in cities.columns]
cities = cities.drop(columns=["COUNTRY_EXT_ID","external_id"])
cities.to_sql(name="cities",con=engine,if_exists="append",index=False)



2